# 03 langchain 提示词
https://python.langchain.com/docs/modules/model_io/prompts/

任何语言模型应用程序的核心元素都是...模型。
LangChain为您提供了与任何语言模型交互的构建块。
Prompts：模板化、动态选择和管理模型输入
语言模型：通过通用接口调用语言模型
输出解析器：从模型输出中提取信息
<img src="https://python.langchain.com/assets/images/model_io-1f23a36233d7731e93576d6885da2750.jpg" alt="图片" width="900" height="500">




### 3.1 Prompts
语言模型提示是用户提供的一组指令或输入，用于指导模型的响应，帮助它理解上下文并生成相关且连贯的基于语言的输出，例如回答问题、完成句子或参与对话。
LangChain 提供了多个类和函数来帮助构造和使用提示。
提示模板：参数化模型输入
示例选择器：动态选择要包含在提示中的示例

#### 3.1.1 PromptTemplate
提示模板是为语言模型生成提示的预定义配方。
模板可能包括说明、几个镜头示例以及适合给定任务的特定上下文和问题。
LangChain 提供了创建和使用提示模板的工具。
LangChain 致力于创建与模型无关的模板，以便轻松地跨不同语言模型重用现有模板。
通常，语言模型希望提示是字符串或聊天消息列表。

In [1]:
from langchain import PromptTemplate #用于 PromptTemplate 为字符串提示创建模板。

#默认情况下， PromptTemplate 使用 Python 的 str.format 语法进行模板化;但是可以使用其他模板语法（例如， jinja2 ）
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [2]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
"Tell me a joke"
)
prompt_template.format()#该模板支持任意数量的变量，包括无变量：

'Tell me a joke'

In [3]:
from langchain import PromptTemplate

#对于其他验证，请显式指定 input_variables 。
#在实例化期间，这些变量将与模板字符串中存在的变量进行比较，如果不匹配，则会引发异常;
invalid_prompt = PromptTemplate(
    input_variables=["adjective"],
    template="Tell me a {adjective} joke about {content}."
)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. 'content' (type=value_error)

In [5]:
#聊天模型的提示是聊天消息列表。
#每条聊天消息都与内容相关联，以及一个名为 role 的附加参数。例如，在 OpenAI 聊天完成 API 中，聊天消息可以与 AI 助手、人员或系统角色相关联。

from langchain.prompts import ChatPromptTemplate

#ChatPromptTemplate.from_messages 接受各种消息表示形式。
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False),
 HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]

In [7]:
#除了使用上一个代码块中使用的（类型、内容）的 2 元组表示形式外，
#您还可以传入 or BaseMessage 的 MessagePromptTemplate 实例。
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatOpenAI

#设置代理
import os
os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'

llm = ChatOpenAI()
llm(template.format_messages(text='i dont like eating tasty things.'))

AIMessage(content='I absolutely love indulging in mouthwatering delicacies!', additional_kwargs={}, example=False)

#### 3.1.2 自定义提示模板
LangChain 提供了一组默认的提示模板，可用于为各种任务生成提示。但是，在某些情况下，默认提示模板可能无法满足您的需求。例如，您可能希望创建一个提示模板，其中包含语言模型的特定动态说明。在这种情况下，您可以创建自定义提示模板。

假设我们希望LLM生成给定名称的函数的英语解释。为了完成此任务，我们将创建一个自定义提示模板，该模板将函数名称作为输入，并格式化提示模板以提供函数的源代码。

In [8]:
import inspect

#我们将创建一个自定义提示模板，该模板将函数名称作为输入，并设置提示格式以提供函数的源代码。为了实现这一点，让我们首先创建一个函数，该函数将返回给定其名称的函数的源代码。
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

def test_add():
    return 1 + 1

In [9]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

PROMPT = """\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"

In [30]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "test_add"
prompt_1 = fn_explainer.format(function_name=test_add)
print(prompt_1)

Given the function name and source code, generate an English language explanation of the function.
Function Name: test_add
Source Code:
def test_add():
    return 1 + 1

Explanation:



In [31]:
from langchain import LLMChain
chat_prompt = ChatPromptTemplate.from_messages([prompt_1])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt
)
chain.run({})

'The function "test_add" takes no input parameters and returns the result of adding 1 and 1 together.'

### 3.1.3 带例子的提示词模板

我们在写提示词模板的时候，有些情况下，我们需要给出一些例子，这样可以让模型更好的理解我们的意图。



In [1]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

#这里我们用字典来表示一个例子，每个示例都应该是一个字典，其中键是输入变量，值是这些输入变量的值。
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

In [2]:

#配置一个格式化程序，该格式化程序将几个镜头示例格式化为字符串。此格式化程序应该是一个 PromptTemplate 对象。
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [3]:
#我们可以使用FewShotPromptTemplate来创建一个提示词模板，该模板将输入变量作为输入，并将其格式化为包含示例的提示词。
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

#### 3.1.3.1 使用例子选择器

在上面的例子中，我们使用了一个简单的例子选择器，它只是返回全部的例子。但是，我们可以使用自定义的例子选择器来选择例子。例如，我们可以使用一个例子选择器，该选择器将选择与输入最相似的例子。

In [5]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
#设置代理
import os
os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    #zh:这是可供选择的示例列表。
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    #zh:这是用于生成嵌入的嵌入类，这些嵌入用于测量语义相似性。
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    #zh:这是用于存储嵌入并进行相似性搜索的 VectorStore 类。
    Chroma,
    # This is the number of examples to produce.
    #zh:这是要生成的示例数。
    k=1
)

# Select the most similar example to the input.
#zh:选择与输入最相似的示例。
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


In [6]:
#我们可以使用 FewShotPromptTemplate 来创建一个提示词模板，该模板将输入变量作为输入，并将其格式化为包含示例的提示词。
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


#### 3.1.3.2 带例子的聊天提示词模板

In [7]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

#zh:这是一个聊天提示词模板，它将输入变量作为输入，并将其格式化为包含示例的提示词。
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

# This is a prompt template used to format each individual example.
# zh:这是一个提示词模板，用于格式化每个单独的示例。
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [10]:
#最后，组装您的最终提示并将其与模型一起使用。
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are wonderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI()

chain.invoke({"input": "What's the square of a triangle?"})

AIMessage(content='A triangle does not have a square because a square is a specific type of quadrilateral with all sides of equal length and all angles of 90 degrees.', additional_kwargs={}, example=False)

#### 3.1.3.3 使用例子选择器的聊天提示词模板


In [11]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

#由于我们使用向量存储来根据语义相似性选择示例，因此我们需要首先填充存储。
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [12]:
to_vectorize

['2+2 4',
 '2+3 5',
 '2+4 6',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [21]:
vectorstore.search('cow say to the moon', search_type='similarity')

[Document(page_content='What did the cow say to the moon? nothing at all', metadata={'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}),
 Document(page_content='Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?', metadata={'input': 'Write me a poem about the moon', 'output': 'One for the moon, and one for me, who are we to talk about the moon?'}),
 Document(page_content='2+2 4', metadata={'input': '2+2', 'output': '4'}),
 Document(page_content='2+4 6', metadata={'input': '2+4', 'output': '6'})]

In [22]:
#创建向量库后，可以创建 example_selector .在这里，我们将不建议仅获取前 2 个示例。
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
# zh:提示词模板将通过将输入传递给 `select_examples` 方法来加载示例
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2+4', 'output': '6'}]

In [23]:
#最后，我们可以使用 FewShotChatMessagePromptTemplate 来创建一个提示词模板，该模板将输入变量作为输入，并将其格式化为包含示例的提示词。
#使用上面 example_selector 创建的模板组装提示模板。
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

# Define the few-shot prompt.
#zh:定义少量提示。
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    #zh:输入变量选择要传递给 example_selector 的值
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    #zh:定义每个示例的格式。
    # In this case, each example will become 2 messages:
    #zh:在这种情况下，每个示例将成为 2 条消息：
    # 1 human, and 1 AI
    #zh:1 人类和 1 人工智能
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [25]:
#最后，组装您的最终提示并将其与模型一起使用。

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are wonderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [27]:
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=0.0)

chain.invoke({"input": "What's 3+3?"})

AIMessage(content='3 + 3 equals 6.', additional_kwargs={}, example=False)

### 3.1.4 模板格式

## jinja2 vs f-string

### f-string（Python 3.6+）

f-string 是 Python 3.6 以后版本中引入的一种新特性，用于在字符串中插入表达式的值。

- **语法简洁**：f-string 使用花括号 `{}` 来包裹变量或表达式，并可以在其中执行简单的运算。
- **性能较好**：与其他字符串格式化方法相比，f-string 通常有更好的性能。
- **仅限于 Python**：f-string 是 Python 的特定功能，不适用于其他编程语言。

```python
name = "Alice"
age = 30
print(f"Hello, {name}! You are {age} years old.")
```

### jinja2

jinja2 是一个流行的模板引擎，常用于网页开发，特别是与 Flask 和 Django 等框架结合使用。

- **功能强大**：jinja2 不仅支持变量替换，还提供了许多控制结构（例如循环和条件语句）以及自定义过滤器和宏等高级功能。
- **可在多种语境下使用**：虽然 jinja2 通常用于 web 开发，但也可以用于其他需要模板功能的场合。
- **需要额外库**：与 f-string 不同，使用 jinja2 需要安装相应的库。

```jinja2
{% for item in items %}
  {{ item.name }}: {{ item.price | currency }}
{% endfor %}
```

总结：
如果你只需要基本的字符串插值和格式化，f-string 可能是更好的选择，因为它的语法简洁且无需额外依赖。
如果你需要更复杂的模板功能（例如循环、条件、自定义过滤器等），jinja2 可能更合适。



In [1]:
#要使用 jinja2 模板：
from langchain.prompts import PromptTemplate

jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="chickens")
# Output: Tell me a funny joke about chickens.

'Tell me a funny joke about chickens'

In [2]:
#要使用 Python f 字符串模板，请执行以下操作：
from langchain.prompts import PromptTemplate

fstring_template = """Tell me a {adjective} joke about {content}"""
prompt = PromptTemplate.from_template(fstring_template)

prompt.format(adjective="funny", content="chickens")
# Output: Tell me a funny joke about chickens.

'Tell me a funny joke about chickens'

### 3.1.5 MessagePromptTemplate的类型

LangChain提供不同类型的 MessagePromptTemplate .最常用的是 AIMessagePromptTemplate 和 SystemMessagePromptTemplate HumanMessagePromptTemplate ，它们分别创建 AI 消息、系统消息和人类消息。
但是，如果聊天模型支持使用任意角色接收聊天消息，则可以使用 ChatMessagePromptTemplate ，它允许用户指定角色名称。

In [4]:
from langchain.prompts import ChatMessagePromptTemplate

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="Jedi", template=prompt)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', additional_kwargs={}, role='Jedi')

In [7]:
#LangChain 还提供了 MessagesPlaceholder ，它使您可以完全控制在格式化过程中要呈现的消息。
# 当您不确定应为邮件提示模板使用什么角色时，或者当您希望在格式化过程中插入邮件列表时，这可能很有用。
from langchain.prompts import MessagesPlaceholder,HumanMessagePromptTemplate,ChatPromptTemplate

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="conversation"), human_message_template])

In [11]:
from langchain.schema import HumanMessage,AIMessage
human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
""")

chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()

[HumanMessage(content='What is the best way to learn programming?', additional_kwargs={}, example=False),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience', additional_kwargs={}, example=False),
 HumanMessage(content='Summarize our conversation so far in 10 words.', additional_kwargs={}, example=False)]